In [1]:
import json

with open("outputs/legal_agent_output.json") as f:
    legal_output = json.load(f)

with open("outputs/compliance_agent_output.json") as f:
    compliance_output = json.load(f)

with open("outputs/finance_agent_output.json") as f:
    finance_output = json.load(f)

with open("outputs/operations_agent_output.json") as f:
    operations_output = json.load(f)

print("✓ Agent outputs loaded")


✓ Agent outputs loaded


In [2]:
ROUTING_RULES = {
    "legal": ["termination", "governing law", "jurisdiction"],
    "compliance": ["gdpr", "audit", "regulatory", "data protection"],
    "finance": ["payment", "fee", "penalty", "invoice"],
    "operations": ["deliverable", "timeline", "sla", "milestone"]
}


In [3]:
ROUTING_RULES["legal"].append("indemnity")


In [4]:
def route_query(query: str):
    query = query.lower()
    selected_agents = []

    for agent, keywords in ROUTING_RULES.items():
        for kw in keywords:
            if kw in query:
                selected_agents.append(agent)
                break

    return selected_agents


In [5]:
print(route_query("Review indemnity and termination clauses"))
print(route_query("Check GDPR compliance"))
print(route_query("Check late payment penalties"))


['legal']
['compliance']
['finance']


In [6]:
def coordinator_execute(query):
    agents = route_query(query)
    results = {}

    for agent in agents:
        if agent == "legal":
            results["legal"] = legal_output
        elif agent == "compliance":
            results["compliance"] = compliance_output
        elif agent == "finance":
            results["finance"] = finance_output
        elif agent == "operations":
            results["operations"] = operations_output

    return results


In [7]:
query = "Review termination, GDPR compliance, payment terms, and SLAs"
coordinator_result = coordinator_execute(query)

print(coordinator_result.keys())


dict_keys(['legal', 'compliance', 'finance', 'operations'])


In [8]:
!pip install langgraph


In [9]:
from langgraph.graph import StateGraph, END
from typing import TypedDict


In [10]:
class GraphState(TypedDict):
    query: str
    legal: dict
    compliance: dict
    finance: dict
    operations: dict


In [11]:
def compliance_node(state: GraphState):
    print("Compliance Agent running")
    state["compliance"] = compliance_output
    return state

def legal_node(state: GraphState):
    print("Legal Agent running")
    state["legal"] = legal_output
    return state

def finance_node(state: GraphState):
    print("Finance Agent running")
    state["finance"] = finance_output
    return state

def operations_node(state: GraphState):
    print("Operations Agent running")
    state["operations"] = operations_output
    return state


In [12]:
graph = StateGraph(GraphState)


In [13]:
graph.add_node("compliance_agent", compliance_node)
graph.add_node("legal_agent", legal_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)


In [14]:
graph.set_entry_point("compliance_agent")
graph.add_edge("compliance_agent", "legal_agent")
graph.add_edge("legal_agent", "finance_agent")
graph.add_edge("finance_agent", "operations_agent")
graph.add_edge("operations_agent", END)


In [15]:
app = graph.compile()

input_state = {
    "query": "Review termination, GDPR compliance, payment terms, and SLAs",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(input_state)
print(result.keys())


Compliance Agent running
Legal Agent running
Finance Agent running
Operations Agent running
dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])


In [16]:
def conditional_router(state: GraphState):
    q = state["query"].lower()

    if "termination" in q or "indemnity" in q:
        return "legal_agent"
    if "gdpr" in q or "audit" in q:
        return "compliance_agent"
    if "payment" in q or "penalty" in q:
        return "finance_agent"
    if "sla" in q or "milestone" in q:
        return "operations_agent"

    return "legal_agent"


In [17]:
graph = StateGraph(GraphState)

graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)

graph.set_conditional_entry_point(conditional_router)

graph.add_edge("legal_agent", END)
graph.add_edge("compliance_agent", END)
graph.add_edge("finance_agent", END)
graph.add_edge("operations_agent", END)

app = graph.compile()


In [18]:
app.invoke({
    "query": "Review termination clause",
    "legal": {}, "compliance": {}, "finance": {}, "operations": {}
})

app.invoke({
    "query": "Check late payment penalties",
    "legal": {}, "compliance": {}, "finance": {}, "operations": {}
})

app.invoke({
    "query": "Check GDPR compliance and payment terms",
    "legal": {}, "compliance": {}, "finance": {}, "operations": {}
})


Legal Agent running
Finance Agent running
Compliance Agent running


{'query': 'Check GDPR compliance and payment terms',
 'legal': {},
 'compliance': {'agent_name': 'Compliance Agent',
  'model': 'google/gemma-2b-it',
  'timestamp': '2025-12-29T18:04:16.940875',
  'input_summary': {'num_context_chunks': 12,
   'combined_text_length': 8761,
   'source_queries': ['What are the data protection and privacy obligations?',
    'What regulatory requirements must be followed?',
    'What are the audit and reporting requirements?',
    'What are the confidentiality and non-disclosure obligations?']},
  'output': {'clause_type': 'Compliance Analysis',
   'extracted_clauses': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."],
   'risk_level': 'high',
   'confidence': 1.0,
   'evidence': ["The clause clearly requires the receiving party to maintain confidentiality of the other party's confidential information."]},
  'raw_response': '{\n  "extracted_clauses": [\

In [19]:
from typing import TypedDict, List, Dict, Any

class GraphState(TypedDict):
    query: str
    memory: List[Dict[str, Any]]
    validation_notes: List[str]
    legal: dict
    compliance: dict
    finance: dict
    operations: dict


In [20]:
input_state = {
    "query": "Review termination, GDPR compliance, payment terms, and SLAs",
    "memory": [],
    "validation_notes": [],
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}


In [21]:
def compliance_node(state: GraphState):
    output = compliance_output
    state["compliance"] = output

    state["memory"].append({
        "agent": "compliance",
        "findings": output["output"]["extracted_clauses"]
    })

    print(" Memory after Compliance Agent:")
    print(state["memory"])

    return state


In [22]:
def finance_node(state: GraphState):
    compliance_findings = [
        m for m in state["memory"] if m["agent"] == "compliance"
    ]

    output = finance_output
    state["finance"] = output

    if compliance_findings:
        state["validation_notes"].append(
            "Finance reviewed compliance findings for penalty conflicts."
        )

    state["memory"].append({
        "agent": "finance",
        "findings": output["output"]["extracted_clauses"]
    })

    print("Memory after Finance Agent:")
    print(state["memory"])

    return state


In [23]:
def legal_node(state: GraphState):
    output = legal_output
    state["legal"] = output

    state["memory"].append({
        "agent": "legal",
        "findings": output["output"]["extracted_clauses"]
    })

    print(" Memory after Legal Agent:")
    print(state["memory"])

    return state


In [24]:
from langgraph.graph import StateGraph, END

graph = StateGraph(GraphState)

graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("legal_agent", legal_node)

graph.set_entry_point("compliance_agent")
graph.add_edge("compliance_agent", "finance_agent")
graph.add_edge("finance_agent", "legal_agent")
graph.add_edge("legal_agent", END)

app = graph.compile()
result = app.invoke(input_state)
print(result)


 Memory after Compliance Agent:
[{'agent': 'compliance', 'findings': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."]}]
Memory after Finance Agent:
[{'agent': 'compliance', 'findings': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."]}, {'agent': 'finance', 'findings': ['In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision of any Service, including, without limitation, license fees and payments to third-party service providers or subcontractors (such included expenses, collectively, ', '), Recipient shall reimburse Provider for all such Out-of-Pocket Costs.', 'Provider shall provide Recipient with monthly invo']}]
 Memory after Legal Agent:
[{'agent': 'compliance', 'findings': ["The receiving party will not disclose the other party'

In [25]:
print("\n FINAL MEMORY:")
for m in result["memory"]:
    print(m)

print("\n VALIDATION NOTES:")
print(result["validation_notes"])



 FINAL MEMORY:
{'agent': 'compliance', 'findings': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."]}
{'agent': 'finance', 'findings': ['In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision of any Service, including, without limitation, license fees and payments to third-party service providers or subcontractors (such included expenses, collectively, ', '), Recipient shall reimburse Provider for all such Out-of-Pocket Costs.', 'Provider shall provide Recipient with monthly invo']}
{'agent': 'legal', 'findings': ["The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.", 'The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination.']}

 VALIDATION

In [26]:
def operations_node(state: GraphState):
    legal_findings = [
        m for m in state["memory"] if m["agent"] == "legal"
    ]

    output = operations_output
    state["operations"] = output

    if legal_findings:
        state["validation_notes"].append(
            "Operations validated SLA enforceability based on legal findings."
        )

    state["memory"].append({
        "agent": "operations",
        "findings": output["output"]["extracted_clauses"]
    })

    print("🧠 Memory after Operations Agent:")
    print(state["memory"])

    return state


In [27]:
COMPLIANCE_QUERY = """
Identify clauses related to:
- Regulatory compliance
- Data protection
- Audits and reporting
- GDPR
"""


In [28]:
compliance_pipeline_output = {
    "compliance_analysis": compliance_output
}


In [29]:
FINANCE_QUERY = """
Identify clauses related to:
- Payment terms
- Penalties
- Late fees
- Interest
"""


In [30]:
finance_pipeline_output = {
    "finance_analysis": finance_output
}


In [31]:
LEGAL_QUERY = """
Identify clauses related to:
- Termination
- Governing law
- Jurisdiction
- Indemnification
"""


In [32]:
legal_pipeline_output = {
    "legal_analysis": legal_output
}


In [33]:
OPERATIONS_QUERY = """
Identify clauses related to:
- Deliverables
- Timelines
- Milestones
- SLAs
- Uptime
"""


In [34]:
operations_pipeline_output = {
    "operations_analysis": operations_output
}


In [35]:
def coordinator_merge(legal, compliance, finance, operations):
    return {
        "legal": legal["legal_analysis"],
        "compliance": compliance["compliance_analysis"],
        "finance": finance["finance_analysis"],
        "operations": operations["operations_analysis"]
    }

merged_output = coordinator_merge(
    legal_pipeline_output,
    compliance_pipeline_output,
    finance_pipeline_output,
    operations_pipeline_output
)


In [36]:
RISK_SCORE = {"low": 1, "medium": 2, "high": 3}

def compute_overall_risk(merged):
    risks = []
    confidences = []
    clauses = []

    for section in merged.values():
        risk = section["output"]["risk_level"].lower()
        risks.append(RISK_SCORE.get(risk, 0))
        confidences.append(section["output"]["confidence"])
        clauses.extend(section["output"]["extracted_clauses"])

    return {
        "overall_risk": max(risks),
        "average_confidence": round(sum(confidences) / len(confidences), 2),
        "highest_risk_clause": clauses[0] if clauses else None
    }

overall_summary = compute_overall_risk(merged_output)


In [37]:
final_output = {
    "merged_analysis": merged_output,
    "summary": overall_summary
}

print(final_output)


{'merged_analysis': {'legal': {'agent_name': 'Legal Agent', 'model': 'google/gemma-2b-it', 'timestamp': '2025-12-29T18:03:41.231900', 'input_summary': {'num_context_chunks': 25, 'combined_text_length': 18562, 'source_queries': ['What is the governing law and jurisdiction?', 'What happens in case of breach of contract?', 'What are the termination clauses and conditions?', 'What are the payment terms and conditions?', 'What are the confidentiality and non-disclosure obligations?']}, 'output': {'clause_type': 'Legal Analysis', 'extracted_clauses': ["The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.", 'The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination.'], 'risk_level': 'low', 'confidence': 0.85, 'evidence': ['The first sentence of the clause describes different events that can lead to